In [0]:
"""#Keycolumns
dbutils.widgets.text("keycols","['flight_id']")

#CDC column

dbutils.widgets.text("cdccol","")

#Backed data refresh
dbutils.widgets.text("backdated_refresh","")

#source object

dbutils.widgets.text("source_object","")


# Source Schema

dbutils.widgets.text("source_schema","")"""


In [0]:
"""key_cols_list = eval(dbutils.widgets.get("keycols"))

cdc_col = dbutils.widgets.get("cdccol")

backdated_refresh = dbutils.widgets.get("backdated_refresh")

source_object = dbutils.widgets.get("source_object")

source_schema = dbutils.widgets.get("source_schema") """




incremental data ingestion


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

Parameters

In [0]:
"""catalog = "workspace"

key_cols = "['flight_id']"
key_cols_list = eval(key_cols)

cdc_col = "modified_date"

backdated_refresh = ""

source_object = "silver_flights"

source_schema = "silver"

target_object = "Dimflights"

target_schema = "gold"

surrogate_key = "Dimflightskey"
"""

do changes by changing values of parameters for different column

In [0]:
catalog = "workspace"

key_cols = "['passenger_id']"
key_cols_list = eval(key_cols)

cdc_col = "modified_date"

backdated_refresh = ""

source_object = "silver_passengers"

source_schema = "silver"

target_object = "Dimpassengers"

target_schema = "gold"

surrogate_key = "Dimpassengerskey"


Last load Data

In [0]:
if len(backdated_refresh) == 0:
    if spark.catalog.tableExists(f"workspace.{target_schema}.{target_object}"):
        last_load = spark.sql(f"Select max({cdc_col}) from workspace.{target_schema}.{target_object}").collect[0][0]
    else:
        last_load = "1900-01-01 00:00:00"
else: 
    last_load = backdated_refresh 

In [0]:
last_load

In [0]:
df_src = spark.sql(f"SELECT * FROM workspace.{source_schema}.{source_object} WHERE {cdc_col} >= '{last_load}'")


 Old vs New Records

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    key_cols_string_incremental = ", ".join(key_cols_list)
    df_trg = spark.sql(f"SELECT {key_cols_string_incremental},{surrogate_key},create_date,update_date FROM {catalog}.{target_schema}.{target_object}")
else:
    key_cols_string_init = [f"'' AS `{i}`" for i in key_cols_list]  # Use backticks for column aliases
    key_cols_string_init = ", ".join(key_cols_string_init)
    df_trg = spark.sql(f"SELECT {key_cols_string_init}, CAST('0' AS INT) AS {surrogate_key}, CAST('1900-01-01 00:00:00' AS timestamp) AS create_date, CAST('1900-01-01 00:00:00' AS timestamp) AS update_date WHERE 1 = 0")

In [0]:
df_trg.display()

In [0]:
key_cols_string = ", ".join(key_cols_list)


join condition

In [0]:
join_condition = " AND ".join([f"src.{i} = trg.{i}" for i in key_cols_list])


In [0]:
  df_src.createOrReplaceTempView("src")
  df_trg.createOrReplaceTempView("trg")
  df_join = spark.sql(f"""
  SELECT
    src.*,
    trg.{surrogate_key} ,
    trg.create_date ,
    trg.update_date 
  FROM
    src
    LEFT JOIN trg""")

In [0]:
df_join.display()

In [0]:
df_old = df_join.filter(col(f'{surrogate_key}').isNotNull())
df_new = df_join.filter(col(f'{surrogate_key}').isNull())


In [0]:
df_new.display()

Preparing DF_OLD


In [0]:
df_old_enr = df_old.withColumn('update_date',current_timestamp())


Preparing DF_NEW

In [0]:
df_new.display()

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    max_surrogate_key = spark.sql(f"SELECT MAX({surrogate_key}) FROM {catalog}.{target_schema}.{target_object}").collect()[0][0]
    df_new_enr = df_new.withColumn(f'{surrogate_key}',lit(max_surrogate_key)+lit(1)+monotonically_increasing_id())\
    .withColumn('create_date',current_timestamp())\
    .withColumn('update_date',current_timestamp())
else:
    max_surrogate_key = 0
    df_new_enr = df_new.withColumn(f'{surrogate_key}',lit(max_surrogate_key)+lit(1)+monotonically_increasing_id())\
    .withColumn('create_date',current_timestamp())\
    .withColumn('update_date',current_timestamp())

In [0]:
df_old_enr.display()


Union

In [0]:
df_union = df_old_enr.unionByName(df_new_enr)


In [0]:
df_union.display()


UPSERT

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_union.alias("src"), f"trg.{surrogate_key} = src.{surrogate_key}")\
    .whenMatchedUpdateAll(condition=f"src.{cdc_cols} >= trg.{cdc_cols}")\
    .whenNotMatchedInsertAll()\
    .execute()
else:
    df_union.write.format("delta")\
    .mode("append")\
    .saveAsTable(f"{catalog}.{target_schema}.{target_object}")